In [1]:
import brightway2 as bw
import bw2data as bd
import bw2analyzer as ba

#reduce?
import ast
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import dopo
from dopo import*

In [2]:
bd.projects.set_current("premise-validation-try1")
bw.bw2setup()

bio3=bw.Database('biosphere3')
ei39=bw.Database('ecoinvent 3.9.1 cutoff')
ei39SSP2=bw.Database('ei_cutoff_3.9_image_SSP2-RCP19_2050 2024-06-27')

Biosphere database already present!!! No setup is needed


FILTERS

In [3]:
#sector filters file names/paths

cement = 'cement_small.yaml'
electricity = 'electricity_small.yaml'
fuels= 'fuels_small.yaml'
steel = 'steel_small.yaml'
transport = 'transport_small.yaml'

In [4]:
files_dict={}
files_dict['Cement']={'yaml': 'yamls\cement_small.yaml', 
                      'yaml identifier': 'Cement'}
files_dict['Steel']= {'yaml':'yamls\steel_small.yaml',
                            'yaml identifier': 'Steel'} #yaml identifier is the name of the filter in the yaml file, in the first line.
files_dict

{'Cement': {'yaml': 'yamls\\cement_small.yaml', 'yaml identifier': 'Cement'},
 'Steel': {'yaml': 'yamls\\steel_small.yaml', 'yaml identifier': 'Steel'}}

In [5]:
import dopo.filter_sectors

#for plot 1 and 2
dictionary_one = dopo.filter_sectors.process_yaml_files(files_dict, ei39SSP2)

#for comparison
premise_dict = dopo.filter_sectors.process_yaml_files(files_dict, ei39SSP2)
ecoinvent_dict = dopo.filter_sectors.process_yaml_files(files_dict, ei39)


Processing Cement with database ei_cutoff_3.9_image_SSP2-RCP19_2050 2024-06-27
Activities for Cement:
  ('ei_cutoff_3.9_image_SSP2-RCP19_2050 2024-06-27', 'f8b84f45f50d3bd7ff4feaabdb493f6a')
  ('ei_cutoff_3.9_image_SSP2-RCP19_2050 2024-06-27', 'df49e8f525497f2fbd56bcdc80ff0cde')
  ('ei_cutoff_3.9_image_SSP2-RCP19_2050 2024-06-27', '3c16b45db40210cd97de6574b2f47aaf')
  ('ei_cutoff_3.9_image_SSP2-RCP19_2050 2024-06-27', 'a3c2064d83411f7963af550c04c869a1')
  ('ei_cutoff_3.9_image_SSP2-RCP19_2050 2024-06-27', 'fcb666edf2a01467e555eeff5b4a5bbb')
  ('ei_cutoff_3.9_image_SSP2-RCP19_2050 2024-06-27', '86841f8c7ee2668f244d3b8e34f41932')
  ('ei_cutoff_3.9_image_SSP2-RCP19_2050 2024-06-27', '36a53c174f34e672bc15b7e55563685e')
Processing Steel with database ei_cutoff_3.9_image_SSP2-RCP19_2050 2024-06-27
Activities for Steel:
  ('ei_cutoff_3.9_image_SSP2-RCP19_2050 2024-06-27', 'af6bd1221fc0206541fbaf481397bf0d')
  ('ei_cutoff_3.9_image_SSP2-RCP19_2050 2024-06-27', '18b0dcf01dd401e1549b3796e3786213

In [6]:
finder=dopo.methods.MethodFinder()

finder.find_and_create_method(criteria=['IPCC', '2013', 'GWP100'], exclude=['no LT'])
finder.find_and_create_method(criteria=['EN15804','Cumulative', 'non-renewable' ])
finder.find_and_create_method(criteria=['land occupation','selected'])
finder.find_and_create_method(criteria=['EN15804','fresh water'])
method_dict=finder.get_all_methods()
method_dict

{'method_1': {'object': Brightway2 Method: IPCC 2013: climate change: global warming potential (GWP100),
  'method name': ('IPCC 2013',
   'climate change',
   'global warming potential (GWP100)'),
  'short name': 'global warming potential (GWP100)',
  'unit': 'kg CO2-Eq'},
 'method_2': {'object': Brightway2 Method: EN15804: inventory indicators ISO21930: Cumulative Energy Demand - non-renewable energy resources,
  'method name': ('EN15804',
   'inventory indicators ISO21930',
   'Cumulative Energy Demand - non-renewable energy resources'),
  'short name': 'Cumulative Energy Demand - non-renewable energy resources',
  'unit': 'megajoule'},
 'method_3': {'object': Brightway2 Method: selected LCI results: resource: land occupation,
  'method name': ('selected LCI results', 'resource', 'land occupation'),
  'short name': 'land occupation',
  'unit': 'square meter-year'},
 'method_4': {'object': Brightway2 Method: EN15804: inventory indicators ISO21930: use of net fresh water,
  'method na

In [7]:
scores_dictionary_one = dopo.sector_lca_scores(dictionary_one, method_dict, cutoff=0.2)

Omitting activity name common prefix: 'cement production, '
Omitting activity name common prefix: 'cement production, '
Omitting activity name common prefix: 'cement production, '
Omitting activity name common prefix: 'cement production, '
Omitting activity name common prefix: 'steel production, electric, '
Omitting activity name common prefix: 'steel production, electric, '
Omitting activity name common prefix: 'steel production, electric, '
Omitting activity name common prefix: 'steel production, electric, '


In [8]:
import re
from openpyxl import load_workbook

def _add_statistics(df, column_name='total'):

    '''
    It is called in the function sector_lca_scores_to_excel_and_column_positions

    It adds statistical indicators to a dataframe based on total column which are used for plotting.

    returns updated dataframe
    '''
    
    #Need a rank row to plot the total LCA scores in descending order (satter opepyxl function takes in non categorial values)
    df['rank'] = df[column_name].rank(method="first", ascending=False)

    # Calculate mean, standard deviation, and IQR
    df['mean'] = df[column_name].mean()
    df['2std_abv'] = df['mean'] + df[column_name].std() * 2
    df['2std_blw'] = df['mean'] - df[column_name].std() * 2
    df['q1'] = df[column_name].quantile(0.25)
    df['q3'] = df[column_name].quantile(0.75)
    
    # Reorder the columns to place the new columns after 'total'
    cols = df.columns.tolist()
    total_index = cols.index(column_name) + 1
    new_cols = ['rank', 'mean', '2std_abv', '2std_blw', 'q1', 'q3']
    cols = cols[:total_index] + new_cols + cols[total_index:-len(new_cols)]
    
    return df[cols]


def _find_first_input_column(df):
    '''
    It is called in the function sector_lca_scores_to_excel_and_column_positions. Needs to be called before _clean_column_labels function.
    Detects the first column in the dataframe which contains input contribution data and saves its index. 
    This is relevant for calling the right column for defining the to be plotted data dynamically as not all dataframes have the same column order (some contain "direct emissions" for instance).
    '''
    
    def _clean_label(label):
        return label if label is not None else 'Unnamed'
    
    # Apply the cleaning function to all column names
    df.columns = [_clean_label(col) for col in df.columns]
    
    # Regular expression pattern to match "Number: Name"
    pattern = r'^\d+:\s*'
    
    for idx, column in enumerate(df.columns):
        if (column is not None and re.match(pattern, column)) or column == 'Unnamed' or column == 'direct emissions':
            return idx

    return None

def _clean_column_labels(df):

    '''
    It is called in the function sector_lca_scores_to_excel_and_column_positions. Needs to be run after _find_first_input_column.

    It removes unnecessary numbers in the column header.

    Returns df with formated column labels.
    '''
    # Function to remove numbers and colon from column names
    def _clean_label(label):
        if label is None:
            return 'Unnamed'  # or return 'Unnamed' if you prefer a placeholder
        return re.sub(r'^\d+:\s*', '', str(label))

    # Apply the cleaning function to all column names
    df.columns = [_clean_label(col) for col in df.columns]

    return df

def _add_sector_marker(df, sector):
    '''
    It is called in the function sector_lca_scores_to_excel_and_column_positions.

    It adds information about the sector for titel and labeling in plotting.

    Returns df with added column.
    '''
    
    # Add sector marker column
    df['sector']=str(sector) # potentially remove!
    # Reorder the columns to move 'sector' after 'product'
    columns = list(df.columns)

    if 'product' in df.columns:
        product_index = columns.index('product')
        # Insert 'sector' after 'product'
        columns.insert(product_index + 1, columns.pop(columns.index('sector')))
    else:
        # If 'product' does not exist, 'sector' remains in the last column
        columns.append(columns.pop(columns.index('sector')))
        
    # Reassign the DataFrame with the new column order
    df = df[columns]
    return df

def _categorize_sheets_by_sector(file_path):
    # Load the workbook
    workbook = load_workbook(filename=file_path, read_only=True)
    
    # Initialize a dictionary to hold sectors and their corresponding sheet names
    worksheet_dict = {}
    
    # Iterate over all sheet names in the workbook
    for sheet_name in workbook.sheetnames:
        # Skip combined sector sheets (assuming these sheets don't have an underscore)
        if '_' not in sheet_name:
            continue
        
        # Split the sheet name to extract the sector (assumes sector is the first part)
        sector = sheet_name.split('_')[0]
        
        # Add the sheet name to the corresponding sector in the dictionary
        if sector in worksheet_dict:
            worksheet_dict[sector].append(sheet_name)
        else:
            worksheet_dict[sector] = [sheet_name]
    
    return worksheet_dict

In [9]:
import pandas as pd

def sector_lca_scores_to_excel(scores_dict, excel_file_name):
    """ 
    What it does:
        - Creates a dataframe for each method and sector from the lca scores dictionary
        - Before storing each df in a worksheet in an excel file it:
                - shortens the column labels of the input (removing cpc code)
                - adds a sector name marker for keeping track in excel (when plotting can use it for labeling)
                - adds statistics for plotting
                - creates a dictionary which holds the indexes to the columns we need to call for plotting, this makes it dynamic. Otherwise need to hardcode index column number for openpxyl.
    What it returns:
        - Returns the index positions dictionary where the key is "sector_method"
        - Creates excel file as defined by user
    """

    # Dictionary to store positions of columns for each method
    column_positions = {}

    # DataFrames to store combined sector data
    combined_sector_dfs = {}
    method_dfs = []

    # Process each sector and its methods
    for sector in scores_dict.keys():
        sector_dfs = []
        lca_scores = scores_dict[sector]['lca_scores']

        # Process each method for the current sector
        for method, table in lca_scores.items():
            df = pd.DataFrame(table)

            # Add sector marker
            df = _add_sector_marker(df, sector)

            # Add statistics to the DataFrame
            df = _add_statistics(df)

            # Get the index values of columns
            columns_of_interest = ["total", "rank", "mean", "2std_abv", "2std_blw", "q1", "q3", "method", "method unit"]
            positions = {col: df.columns.get_loc(col) for col in columns_of_interest if col in df.columns}
            column_positions[f"{sector}_{method}"] = positions
            #column_positions[sector][method]=positions


            # Find the first input column and add it to the positions dictionary
            first_input_col_index = _find_first_input_column(df)
            if first_input_col_index is not None:
                positions["first_input"] = first_input_col_index

            # Store the positions for this method
            column_positions[f"{sector}_{method}"]= positions

            # Remove CPC from input labels
            df = _clean_column_labels(df)

            sector_dfs.append(df)

            # Store method-specific DataFrames for later
            method_dfs.append((f"{sector}_{method}", df))

        # Combine all dataframes for this sector
        combined_df = pd.concat(sector_dfs, axis=0, ignore_index=True, sort=False).fillna(0)
        combined_sector_dfs[sector] = combined_df

    # Write to Excel file
    with pd.ExcelWriter(excel_file_name, engine='openpyxl') as writer:
        # First write all combined sector sheets
        for sector, combined_df in combined_sector_dfs.items():
            worksheet_name_big = f"{sector}"
            if len(worksheet_name_big) > 31:
                worksheet_name_big = worksheet_name_big[:31]
            combined_df.to_excel(writer, sheet_name=worksheet_name_big, index=False)

        # Then write all method-specific sheets
        for worksheet_name, df in method_dfs:
            if len(worksheet_name) > 31:
                worksheet_name = worksheet_name[:31]
            df.to_excel(writer, sheet_name=worksheet_name, index=False)

    return column_positions


In [10]:
from openpyxl import load_workbook
from openpyxl.chart import ScatterChart, Reference, Series

def dot_plots_xcl(filepath_workbook, index_positions):

    worksheet_dict = _categorize_sheets_by_sector(filepath_workbook)
    
    # Load the workbook
    wb = load_workbook(filepath_workbook)
    
    # Iterate over each sector and its associated worksheets
    for sector, worksheet_names in worksheet_dict.items():
        
        # Create or get the chart sheet for the current sector
        chart_sheet_name = f"{sector}_charts"
        if chart_sheet_name in wb.sheetnames:
            ws_charts = wb[chart_sheet_name]
        else:
            ws_charts = wb.create_sheet(chart_sheet_name)        
                
        # Initial position for the first chart
        current_row = 1  # Start placing charts from row 1
        current_col = 1  # Start placing charts from column 1
        chart_height = 30  # Number of rows a chart occupies
        chart_width = 12   # Number of columns a chart occupies
        charts_per_row = 3  # Number of charts per row
        
        # Iterate over each worksheet name in the current sector
        for i, worksheet_name in enumerate(worksheet_names):
            ws = wb[worksheet_name]

            # Find min_row, max_row and max_column
            max_row = ws.max_row
            max_column = ws.max_column
            min_row = 1

            # Find the key in index_positions that contains worksheet_name
            matching_key = None
            for key in index_positions.keys():
                if worksheet_name in key:
                    matching_key = key
                    break

            if not matching_key:
                print(f"Warning: No matching key found for worksheet '{worksheet_name}'. Skipping...")
                continue

            # Retrieve the column positions from the index_positions dictionary
            positions = index_positions[matching_key]
            total_col = positions.get("total", None) + 1
            rank_col = positions.get("rank", None) + 1
            mean_col = positions.get("mean", None) + 1
            std_adv_col = positions.get("2std_abv", None) + 1
            std_blw_col = positions.get("2std_blw", None) + 1
            q1_col = positions.get("q1", None) + 1
            q3_col = positions.get("q3", None) + 1
            method_col = positions.get("method", None) + 1
            method_unit_col = positions.get("method unit", None) + 1
            
            # Ensure that all required columns are present
            if None in [total_col, rank_col, mean_col, std_adv_col, std_blw_col, q1_col, q3_col, method_col, method_unit_col]:
                print(f"Warning: Missing columns in worksheet '{worksheet_name}' for sector '{sector}'. Skipping...")
                continue
            
            # Create a ScatterChart (or other chart type as needed)
            chart = ScatterChart()

            # Chart titles
            method_value = ws.cell(row=2, column=method_col).value
            chart.title = f"{method_value} LCA scores for {sector} sector" 
            
            method_unit_value = ws.cell(row=2, column=method_unit_col).value
            chart.y_axis.title = f"{method_unit_value}"
            chart.x_axis.title = 'activity rank'
            # Avoid overlap
            chart.title.overlay = False
            chart.x_axis.title.overlay = False
            chart.y_axis.title.overlay = False 

            # Define the data range for the chart
            y_values = Reference(ws, min_col=total_col, min_row=min_row, max_row=max_row)
            x_values = Reference(ws, min_col=rank_col, min_row=min_row, max_row=max_row)

            # Create a series and add it to the chart
            series = Series(y_values, x_values, title_from_data=True)
            chart.series.append(series)
            chart.style = 9

            # Customize the series to show only markers (dots)
            series.marker.symbol = "circle"
            series.marker.size = 5
            series.graphicalProperties.line.noFill = True

            # ADJUST X-AXIS
            chart.x_axis.tickLblPos = "low"
            chart.x_axis.majorGridlines = None 
            chart.x_axis.tickMarkSkip = 1  # Show all tick marks, this adresses the tick lines 
            chart.x_axis.tickLblSkip = 1  # Show all labels, doesnt work

            chart.x_axis.scaling.orientation = "minMax"
            chart.x_axis.crosses = "autoZero"
            chart.x_axis.axPos = "b"
            chart.x_axis.delete = False

            # ADJUST Y-AXIS
            chart.y_axis.tickLblPos = "nextTo"  # Position the labels next to the tick marks
            chart.y_axis.delete = False  # Ensure axis is not deleted
            chart.y_axis.number_format = '0.00000'
            chart.y_axis.majorGridlines = None 

            # ADD STATS
            # MEAN
            mean_y = Reference(ws, min_col=mean_col, min_row=min_row, max_row=max_row)
            mean_series = Series(mean_y, x_values, title_from_data="True")
            chart.series.append(mean_series)
            mean_series.marker.symbol = "none"  # No markers, just a line
            mean_series.graphicalProperties.line.solidFill = "FF0000"  # Red line for mean value
            mean_series.graphicalProperties.line.width = 10000  # Set line width (default units are EMUs)

            # IQR
            iqr1 = Reference(ws, min_col=q1_col, min_row=min_row, max_row=max_row)
            iqr3 = Reference(ws, min_col=q3_col, min_row=min_row, max_row=max_row)
            iqr1_series = Series(iqr1, x_values, title_from_data="True")
            iqr3_series = Series(iqr3, x_values, title_from_data="True")
            chart.series.append(iqr1_series)
            chart.series.append(iqr3_series)
            iqr1_series.marker.symbol = "none"  # No markers, just a line
            iqr3_series.marker.symbol = "none"
            iqr1_series.graphicalProperties.line.solidFill = "6082B6"  # Blue line 
            iqr3_series.graphicalProperties.line.solidFill = "6082B6"  
            iqr1_series.graphicalProperties.line.width = 10000  # Set line width (default units are EMUs)
            iqr3_series.graphicalProperties.line.width = 10000  # Set line width (default units are EMUs)

            # STD
            std_abv = Reference(ws, min_col=std_adv_col, min_row=min_row, max_row=max_row)
            std_blw = Reference(ws, min_col=std_blw_col, min_row=min_row, max_row=max_row)
            std_abv_series = Series(std_abv, x_values, title_from_data="True")
            std_blw_series = Series(std_blw, x_values, title_from_data="True")
            chart.series.append(std_abv_series)
            chart.series.append(std_blw_series)
            std_abv_series.marker.symbol = "none"  # No markers, just a line
            std_blw_series.marker.symbol = "none"
            std_abv_series.graphicalProperties.line.solidFill = "FFC300"  # yellow line
            std_blw_series.graphicalProperties.line.solidFill = "FFC300"  
            std_abv_series.graphicalProperties.line.width = 10000  # Set line width (default units are EMUs)
            std_blw_series.graphicalProperties.line.width = 10000  # Set line width (default units are EMUs)

            # Set legend position to the right of the plot area
            chart.legend.position = 'r'  # 'r' for right
            chart.legend.overlay = False

            # Adjust chart dimensions
            chart.width = 20  # Width of the chart
            chart.height = 14  # Height of the chart

            # Calculate the position for this chart
            position = ws_charts.cell(row=current_row, column=current_col).coordinate
            ws_charts.add_chart(chart, position)
            
            # Update position for the next chart
            current_col += chart_width +1 
            if (i + 1) % charts_per_row == 0:  # Move to the next row after placing `charts_per_row` charts
                current_row += chart_height +1
                current_col = 1  # Reset to the first column

        # Move the chart sheet to the first position
        wb._sheets.remove(ws_charts)
        wb._sheets.insert(0, ws_charts)

    wb.save(filepath_workbook)
    return current_row


from openpyxl import load_workbook
from openpyxl.chart import BarChart, Reference

def stacked_bars_xcl(filepath_workbook, index_positions, current_row_dot_plot):

    worksheet_dict = _categorize_sheets_by_sector(filepath_workbook)
    # Load the workbook
    wb = load_workbook(filepath_workbook)
    
    # Iterate over each sector and its associated worksheets
    for sector, worksheet_names in worksheet_dict.items():
        
        # Create or get the chart sheet for the current sector
        chart_sheet_name = f"{sector}_charts"
        if chart_sheet_name in wb.sheetnames:
            ws_charts = wb[chart_sheet_name]
        else:
            ws_charts = wb.create_sheet(chart_sheet_name)
                
        # Initial position for the first chart
        chart_height = 30  # Number of rows a chart occupies
        chart_width = 12   # Number of columns a chart occupies
        current_row = current_row_dot_plot + chart_height # Start placing charts from row where dot plots have left of
        current_col = 1  # Start placing charts from column 1
        charts_per_row = 3  # Number of charts per row
        
        # Iterate over each worksheet name in the current sector
        for i, worksheet_name in enumerate(worksheet_names):
            ws = wb[worksheet_name]

            # Find the key in index_positions that contains worksheet_name
            matching_key = None
            for key in index_positions.keys():
                if worksheet_name in key:
                    matching_key = key
                    break

            if not matching_key:
                print(f"Warning: No matching key found for worksheet '{worksheet_name}'. Skipping...")
                continue

            # Retrieve the column positions from the index_positions dictionary
            positions = index_positions[matching_key]

            # Find min_row, max_row and max_column
            max_row = ws.max_row
            max_column = ws.max_column
            input_min_col = positions.get("first_input", None) + 1
            rank_col = positions.get("rank", None) + 1
            method_col = positions.get("method", None) + 1
            method_unit_col = positions.get("method unit", None) + 1

            chart = BarChart()
            chart.type = "bar"
            chart.style = 2
            chart.grouping = "stacked"
            chart.overlap = 100

            # Chart titles
            method_value = ws.cell(row=2, column=method_col).value
            chart.title = f"{sector} sector inputs contributions to {method_value}"

            method_unit_value = ws.cell(row=2, column=method_unit_col).value
            chart.y_axis.title = f"{method_unit_value}"
            
            chart.x_axis.title = 'activity rank'

            # Avoid overlap
            chart.title.overlay = False
            chart.x_axis.title.overlay = False
            chart.y_axis.title.overlay = False 
            chart.legend.overlay = False

            # Define data
            data = Reference(ws, min_col=input_min_col, min_row=1, max_row=max_row, max_col=max_column)
            cats = Reference(ws, min_col=rank_col, min_row=2, max_row=max_row)

            chart.add_data(data, titles_from_data=True)
            chart.set_categories(cats)
            chart.shape = 4

            # Modify each series in the chart to disable the inversion of negative values 
            for series in chart.series:
                series.invertIfNegative = False

            # y-axis ticks
            chart.y_axis.tickLblPos = "nextTo"
            chart.y_axis.delete = False  # Ensure axis is not deleted
            chart.y_axis.number_format = '0.000'

            # Adjust chart dimensions
            chart.width = 20  # Width of the chart
            chart.height = 14  # Height of the chart

            # Add the chart to the chart worksheet
            # Calculate the position for this chart
            position = ws_charts.cell(row=current_row, column=current_col).coordinate
            ws_charts.add_chart(chart, position)
            
            # Update position for the next chart
            current_col += chart_width +1
            if (i + 1) % charts_per_row == 0:  # Move to the next row after placing `charts_per_row` charts
                current_row += chart_height +1
                current_col = 1  # Reset to the first column

        # Move the chart sheet to the first position
        wb._sheets.remove(ws_charts)
        wb._sheets.insert(0, ws_charts)
        
    wb.save(filepath_workbook)
    return current_row

In [24]:
import pandas as pd

import re
import pandas as pd
from dopo import generate_sets_from_filters
from dopo import compare_activities_multiple_methods
from dopo import small_inputs_to_other_column
import openpyxl
from openpyxl import load_workbook
from openpyxl.chart import ScatterChart, Reference, Series
from openpyxl.chart import BarChart, Reference

def _lca_scores_compare(database_dict, method_dict):
    # Dictionary to store DataFrames for each sector
    sector_dataframes = {}

    # Labels for the DataFrame columns
    labels = [
        "activity",
        "activity key",
        "reference product",
        "location",
        "method",
        "method unit",
        "total",
    ]

    # Loop through each sector in the database_dict
    for sector, sector_data in database_dict.items():
        # Initialize a dictionary to hold DataFrames for each method in the current sector
        method_dataframes = {}

        # Loop through each method in method_dict
        for meth_key, meth_info in method_dict.items():
            data = []  # Initialize a new list to hold data for the current method
            
            # Extract the 'method name' tuple from the current method info
            method_name = meth_info['method name']
            method_short_name=meth_info['short name']
            method_unit = meth_info['unit']

            # Now loop through each activity in the sector
            for act in sector_data['activities']:
                # Ensure the activity is an instance of the expected class
                if not isinstance(act, bd.backends.peewee.proxies.Activity):
                    raise ValueError("`activities` must be an iterable of `Activity` instances")
                
                # Perform LCA calculations
                lca = bw.LCA({act: 1}, method_name)
                lca.lci()
                lca.lcia()
                
                # Collect data for the current activity and method
                data.append([
                    act["name"],
                    act.key,
                    act.get("reference product"),
                    act.get("location", "")[:25],
                    method_short_name,
                    method_unit,
                    lca.score,
                ])
            
            # Convert the data list to a DataFrame and store it in the sector's dictionary
            method_dataframes[method_short_name] = pd.DataFrame(data, columns=labels)

        # Store the method_dataframes dictionary in the sector_dataframes dictionary
        sector_dataframes[sector] = method_dataframes

    # Now `sector_dataframes` is a dictionary where each key is a sector, and the value is another dictionary with method names and their corresponding DataFrames
    return sector_dataframes


import pandas as pd

def _relative_changes_df(database_dict_eco, database_dict_premise, method_dict):

    ecoinvent_scores = _lca_scores_compare(database_dict_eco, method_dict)
    premise_scores = _lca_scores_compare(database_dict_premise, method_dict)

    relative_dict = {}

    # Iterate over sectors
    for sector_key in ecoinvent_scores:
        # Initialize the sector key in the output dictionary
        if sector_key not in relative_dict:
            relative_dict[sector_key] = {}

        # Iterate over methods within the sector
        for method_key in ecoinvent_scores[sector_key]:
            # Check if the method_key exists in both dictionaries to avoid KeyError
            if method_key in premise_scores.get(sector_key, {}):
                # Get the corresponding DataFrames
                df_ei = ecoinvent_scores[sector_key][method_key]
                df_premise = premise_scores[sector_key][method_key]

                #print(df_ei['activity key'])
                #print(df_premise)

                # Split the 'activity key' to extract the second part
                df_ei['activity_code'] = df_ei['activity key'].apply(lambda x: x[1])  # Access the second element of the tuple
                df_premise['activity_code'] = df_premise['activity key'].apply(lambda x: x[1])

                # Merge the two dataframes based on the activity code and method name
                merged_df = pd.merge(df_ei, df_premise, on=['activity_code', 'method'], suffixes=('_ei', '_premise'))

                # Calculate the relative change
                merged_df['relative_change'] = ((merged_df['total_premise'] - merged_df['total_ei']) / merged_df['total_ei']) * 100

                # Store the result in the dictionary
                relative_dict[sector_key][method_key] = merged_df

    return relative_dict

def _add_sector_marker(df, sector):
    '''
    It is called in the function sector_lca_scores_to_excel_and_column_positions.

    It adds information about the sector for titel and labeling in plotting.

    Returns df with added column.
    '''
    
    # Add sector marker column
    df['sector']=str(sector) # potentially remove!
    # Reorder the columns to move 'sector' after 'product'
    columns = list(df.columns)

    if 'product' in df.columns:
        product_index = columns.index('product')
        # Insert 'sector' after 'product'
        columns.insert(product_index + 1, columns.pop(columns.index('sector')))
    else:
        # If 'product' does not exist, 'sector' remains in the last column
        columns.append(columns.pop(columns.index('sector')))
        
    # Reassign the DataFrame with the new column order
    df = df[columns]
    return df




import pandas as pd
from openpyxl import load_workbook

def relative_changes_db(database_dict_eco, database_dict_premise, method_dict, excel_file):
    relative_dict = _relative_changes_df(database_dict_eco, database_dict_premise, method_dict)
    
    # Load existing workbook and get existing sheet names
    try:
        book = load_workbook(excel_file)
        existing_sheets = book.sheetnames
    except FileNotFoundError:
        # If the file does not exist, we will create a new one, so no need to check existing sheets
        existing_sheets = []
    
    column_positions = {}  # stores the indexes of columns for plotting
    
    with pd.ExcelWriter(excel_file, engine='openpyxl', mode='a') as writer:
        for sector in relative_dict.keys():
            relative_changes = relative_dict[sector]
            
            for method, table in relative_changes.items():
                # Create a DataFrame for the current LCA score table
                df = pd.DataFrame(table)

                # Add sector marker
                df = _add_sector_marker(df, sector) #!! ADJUST      

                # Sort the DataFrame by 'relative_change' from largest negative to largest positive
                df = df.sort_values(by='relative_change', ascending=False)

                # Add a 'rank' column based on the 'relative_change', ranking from most negative to least negative
                df['rank'] = df['relative_change'].rank(ascending=False, method='dense').astype(int)
    
                # Get the index values of columns
                columns_of_interest = ["rank", "relative_change", "method", "method unit_ei"]
                positions = {col: df.columns.get_loc(col) for col in columns_of_interest if col in df.columns}
                column_positions[f"{sector}_comparison_{method}"] = positions

                # Generate worksheet name
                worksheet_name = f"{sector}_comparison_{method}"
                if len(worksheet_name) > 31:
                    worksheet_name = worksheet_name[:31]
                
                # Ensure unique sheet name
                original_worksheet_name = worksheet_name
                counter = 1
                while worksheet_name in existing_sheets:
                    worksheet_name = f"{original_worksheet_name}_{counter}"
                    if len(worksheet_name) > 31:
                        worksheet_name = worksheet_name[:31]
                    counter += 1

                # Save the DataFrame to the Excel file in a new worksheet
                df.to_excel(writer, sheet_name=worksheet_name, index=False)
    
    return column_positions


In [12]:
def _categorize_sheets_by_sector_comparison(file_path):
    # Load the workbook
    workbook = load_workbook(filename=file_path, read_only=True)
    
    # Initialize a dictionary to hold sectors and their corresponding sheet names
    worksheet_dict = {}
    
    # Iterate over all sheet names in the workbook
    for sheet_name in workbook.sheetnames:
        # Skip combined sector sheets (assuming these sheets don't have an underscore)
        if '_comparison' not in sheet_name:
            continue
        
        # Split the sheet name to extract the sector (assumes sector is the first part)
        sector = sheet_name.split('_')[0]
        
        # Add the sheet name to the corresponding sector in the dictionary
        if sector in worksheet_dict:
            worksheet_dict[sector].append(sheet_name)
        else:
            worksheet_dict[sector] = [sheet_name]
    
    return worksheet_dict


In [25]:
from openpyxl import load_workbook
from openpyxl.chart import BarChart, Reference
from openpyxl.chart.axis import ChartLines

def barchart_compare_db_xcl(filename, index_positions, current_row_stacked_bar): #, index_positions=None):
      
    worksheet_dict =_categorize_sheets_by_sector_comparison(file_path=filename)
    # Load the workbook and select the sheet
    wb = load_workbook(filename)

    # Iterate over each sector and its associated worksheets
    for sector, worksheet_names in worksheet_dict.items():
        
        # Create or get the chart sheet for the current sector
        chart_sheet_name = f"{sector}_charts"
        if chart_sheet_name in wb.sheetnames:
            ws_charts = wb[chart_sheet_name]
        else:
            ws_charts = wb.create_sheet(chart_sheet_name)  
        
        # Initial position for the first chart
        current_col = 1  # Start placing charts from column 1
        chart_height = 30  # Number of rows a chart occupies
        chart_width = 12   # Number of columns a chart occupies
        charts_per_row = 3  # Number of charts per row
        current_row = current_row_stacked_bar + chart_height
    
        # Iterate over each worksheet name in the current sector
        for i, worksheet_name in enumerate(worksheet_names):
            ws = wb[worksheet_name]

            # Find the key in index_positions that contains worksheet_name
            matching_key = None
            for key in index_positions.keys():
                if worksheet_name in key:
                    matching_key = key
                    break

            if not matching_key:
                print(f"Warning: No matching key found for worksheet '{worksheet_name}'. Skipping...")
                continue

            # Retrieve the column positions from the index_positions dictionary
            positions = index_positions[matching_key]

            # Find min_row, max_row and max_column
            min_col_data = positions.get("relative_change", None) + 1 #15
            rank_col = positions.get("rank", None) + 1 #17
            method_col = positions.get("method", None) + 1 #5
            method_unit_col = positions.get("method unit_ei", None) + 1 #6

            # Create a bar chart
            chart = BarChart()
            chart.type="bar"
            chart.style=2
            chart.overlap= 100

            # Set the data for the chart
            data = Reference(ws, min_col=min_col_data, min_row=2, max_row=ws.max_row)
            categories = Reference(ws, min_col=rank_col, min_row=2, max_row=ws.max_row)
            chart.add_data(data, titles_from_data=False)
            chart.set_categories(categories)

            # Modify each series in the chart to disable the inversion of negative values 
            for series in chart.series:
                series.invertIfNegative = False

            # Y-axis (categories) settings
            chart.y_axis.majorGridlines = ChartLines()
            chart.y_axis.delete = False
            chart.y_axis.tickLblPos = "low"

            # X-axis (values) settings
            chart.x_axis.majorGridlines = None

            # Chart titles
            method_value = ws.cell(row=2, column=method_col).value
            method_unit_value = ws.cell(row=2, column=method_unit_col).value
            chart.title = f"{sector} - Database relatvie comparison for method: {method_value}"


            chart.x_axis.title = "Activity"
            chart.y_axis.title = "Relative Change (%)"

            # Avoid overlap
            chart.title.overlay = False
            chart.x_axis.title.overlay = False
            chart.y_axis.title.overlay = False 
            chart.legend.overlay = False

            # Adjust chart dimensions
            chart.width = 30  # Increased width for better readability
            chart.height = 15

            # Adjust chart dimensions
            chart.width = 20
            chart.height = 14
            # Calculate the position for this chart
            position = ws_charts.cell(row=current_row, column=current_col).coordinate
            ws_charts.add_chart(chart, position)

            # Update position for the next chart
            current_col += chart_width +1 
            if (i + 1) % charts_per_row == 0:  # Move to the next row after placing `charts_per_row` charts
                current_row += chart_height +1
                current_col = 1  # Reset to the first column

        # Move the chart sheet to the first position
        wb._sheets.remove(ws_charts)
        wb._sheets.insert(0, ws_charts)

            # Add the chart to a new worksheet
            # new_sheet = wb.create_sheet(title="LCA Chart")
            # new_sheet.add_chart(chart, "A1")

    # Save the workbook
    wb.save(filename)

    print(f"Results and chart saved to {filename}")

In [26]:
column_positions=sector_lca_scores_to_excel(scores_dictionary_one, '2908_v13.xlsx')
last_row= dot_plots_xcl('2908_v13.xlsx', column_positions)
current_row_stacked_bar=stacked_bars_xcl('2908_v13.xlsx', column_positions, last_row)
column_positions_compare=relative_changes_db(premise_dict, ecoinvent_dict, method_dict, '2908_v13.xlsx')
barchart_compare_db_xcl('2908_v13.xlsx', column_positions_compare, current_row_stacked_bar)

Results and chart saved to 2908_v13.xlsx


In [28]:
column_positions_compare

{'Cement_comparison_global warming potential (GWP100)': {'rank': 16,
  'relative_change': 14,
  'method': 4,
  'method unit_ei': 5},
 'Cement_comparison_Cumulative Energy Demand - non-renewable energy resources': {'rank': 16,
  'relative_change': 14,
  'method': 4,
  'method unit_ei': 5},
 'Cement_comparison_land occupation': {'rank': 16,
  'relative_change': 14,
  'method': 4,
  'method unit_ei': 5},
 'Cement_comparison_use of net fresh water': {'rank': 16,
  'relative_change': 14,
  'method': 4,
  'method unit_ei': 5},
 'Steel_comparison_global warming potential (GWP100)': {'rank': 16,
  'relative_change': 14,
  'method': 4,
  'method unit_ei': 5},
 'Steel_comparison_Cumulative Energy Demand - non-renewable energy resources': {'rank': 16,
  'relative_change': 14,
  'method': 4,
  'method unit_ei': 5},
 'Steel_comparison_land occupation': {'rank': 16,
  'relative_change': 14,
  'method': 4,
  'method unit_ei': 5},
 'Steel_comparison_use of net fresh water': {'rank': 16,
  'relative_ch

In [27]:
index_pos

{'Cement_Cement_global_warming_potential_(gwp100)': {'total': 8,
  'rank': 9,
  'mean': 10,
  '2std_abv': 11,
  '2std_blw': 12,
  'q1': 13,
  'q3': 14,
  'method': 6,
  'method unit': 7,
  'first_input': 15},
 'Cement_Cement_cumulative_energy_demand_-_non-renewable_energy_resources': {'total': 8,
  'rank': 9,
  'mean': 10,
  '2std_abv': 11,
  '2std_blw': 12,
  'q1': 13,
  'q3': 14,
  'method': 6,
  'method unit': 7,
  'first_input': 15},
 'Cement_Cement_land_occupation': {'total': 8,
  'rank': 9,
  'mean': 10,
  '2std_abv': 11,
  '2std_blw': 12,
  'q1': 13,
  'q3': 14,
  'method': 6,
  'method unit': 7,
  'first_input': 15},
 'Cement_Cement_use_of_net_fresh_water': {'total': 8,
  'rank': 9,
  'mean': 10,
  '2std_abv': 11,
  '2std_blw': 12,
  'q1': 13,
  'q3': 14,
  'method': 6,
  'method unit': 7,
  'first_input': 15},
 'Steel_Steel_global_warming_potential_(gwp100)': {'total': 8,
  'rank': 9,
  'mean': 10,
  '2std_abv': 11,
  '2std_blw': 12,
  'q1': 13,
  'q3': 14,
  'method': 6,
  'm